# A.4b - RGB + Synthetic Depth (4-Channel) Training - FIXED

**Experiment:** A.4b  
**Input:** RGB + Synthetic Depth (4-channel RGBD)  
**Objective:** Test fusion of RGB with synthetic depth (Depth-Anything-V2)  
**Classes:** 1 (fresh_fruit_bunch)

## FIX Applied
This version includes proper 4-channel support:
1. Monkey-patch `imread` to read 4 channels
2. Custom `RGBD4ChTrainer` and `RGBD4ChValidator`
3. `channels: 4` in YAML config

## Prerequisites
1. Upload RGB dataset as: `ffb-localization-dataset`
2. Upload Synthetic Depth dataset as: `ffb-synthetic-depth`

## Training Config
- Model: YOLOv11n
- Epochs: 100
- Patience: 30
- Seeds: 42, 123, 456, 789, 101

In [ ]:
# =============================================================================
# Cell 1: Setup & Install
# =============================================================================
!pip install -q ultralytics

import os
import torch
import torch.nn as nn
import numpy as np
import cv2
import shutil
import pandas as pd
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm

os.environ["WANDB_DISABLED"] = "true"

IS_KAGGLE = os.path.exists('/kaggle/input')
print(f"Running on: {'Kaggle' if IS_KAGGLE else 'Local'}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# =============================================================================
# Cell 2: Path Setup
# =============================================================================
RGB_DATASET = Path('/kaggle/input/ffb-localization-dataset/ffb_localization')
DEPTH_DATASET = Path('/kaggle/input/ffb-synthetic-depth')
BASE_PATH = Path('/kaggle/working')
RUNS_PATH = BASE_PATH / 'runs' / 'detect'
RGBD_DATASET = BASE_PATH / 'rgbd_4ch'
KAGGLE_OUTPUT = BASE_PATH / 'kaggleoutput'
KAGGLE_OUTPUT.mkdir(parents=True, exist_ok=True)

print(f"RGB Dataset: {RGB_DATASET} (exists: {RGB_DATASET.exists()})")
print(f"Depth Dataset: {DEPTH_DATASET} (exists: {DEPTH_DATASET.exists()})")
print(f"Output RGBD: {RGBD_DATASET}")

# Verify datasets
for split in ['train', 'val', 'test']:
    rgb_imgs = len(list((RGB_DATASET / 'images' / split).glob('*.png')))
    depth_imgs = len(list((DEPTH_DATASET / 'images' / split).glob('*.png')))
    rgb_lbls = len(list((RGB_DATASET / 'labels' / split).glob('*.txt')))
    print(f"  {split}: RGB={rgb_imgs}, Depth={depth_imgs}, Labels={rgb_lbls}")

In [ ]:
# =============================================================================
# Cell 3: Create 4-Channel RGBD Dataset
# =============================================================================
print("="*60)
print("CREATING 4-CHANNEL RGBD DATASET")
print("="*60)

for split in ['train', 'val', 'test']:
    print(f"\nProcessing {split.upper()}...")
    
    rgb_img_dir = RGB_DATASET / 'images' / split
    depth_img_dir = DEPTH_DATASET / 'images' / split
    rgb_lbl_dir = RGB_DATASET / 'labels' / split
    
    # Create output directories
    rgbd_img_dir = RGBD_DATASET / 'images' / split
    rgbd_lbl_dir = RGBD_DATASET / 'labels' / split
    rgbd_img_dir.mkdir(parents=True, exist_ok=True)
    rgbd_lbl_dir.mkdir(parents=True, exist_ok=True)
    
    # Get RGB files
    rgb_files = sorted(list(rgb_img_dir.glob('*.png')))
    
    for rgb_path in tqdm(rgb_files, desc=f"Creating 4-ch images ({split})"):
        # Load RGB (3 channels)
        rgb = cv2.imread(str(rgb_path))
        if rgb is None:
            print(f"  Warning: Could not read {rgb_path}")
            continue
        
        # Load synthetic depth (1 channel)
        depth_path = depth_img_dir / rgb_path.name
        depth = cv2.imread(str(depth_path), cv2.IMREAD_GRAYSCALE)
        if depth is None:
            print(f"  Warning: Could not read {depth_path}")
            continue
        
        # Resize depth if needed
        if depth.shape[:2] != rgb.shape[:2]:
            depth = cv2.resize(depth, (rgb.shape[1], rgb.shape[0]))
        
        # Combine RGB (3) + Depth (1) = 4 channels
        depth_expanded = depth[:, :, np.newaxis]
        rgbd_4ch = np.concatenate([rgb, depth_expanded], axis=2)
        
        # Save 4-channel image
        output_img_path = rgbd_img_dir / rgb_path.name
        cv2.imwrite(str(output_img_path), rgbd_4ch)
    
    # Copy labels
    label_files = list(rgb_lbl_dir.glob('*.txt'))
    for lbl_path in label_files:
        shutil.copy(str(lbl_path), str(rgbd_lbl_dir / lbl_path.name))
    
    img_count = len(list(rgbd_img_dir.glob('*.png')))
    lbl_count = len(list(rgbd_lbl_dir.glob('*.txt')))
    print(f"  Done: {img_count} images, {lbl_count} labels")

# Verify sample
sample_img = list((RGBD_DATASET / 'images' / 'train').glob('*.png'))[0]
img = cv2.imread(str(sample_img), cv2.IMREAD_UNCHANGED)
print(f"\nSample image shape: {img.shape} (should be H x W x 4)")

In [ ]:
# =============================================================================
# Cell 4: Write YAML Config with channels: 4
# =============================================================================
config_content = f"""# A.4b RGB+Synthetic Depth 4-Channel Dataset
path: {RGBD_DATASET}
train: images/train
val: images/val
test: images/test

nc: 1
channels: 4

names:
  0: fresh_fruit_bunch
"""

config_path = RGBD_DATASET / 'dataset_rgbd_synthetic.yaml'
with open(config_path, 'w') as f:
    f.write(config_content)

print(f"YAML saved: {config_path}")
print("(includes channels: 4)")
print(config_content)

In [ ]:
# =============================================================================
# Cell 5: Monkey-Patch Ultralytics imread for 4-Channel Support
# =============================================================================
import ultralytics.utils.patches as patches

_original_imread = patches.imread

def imread_4ch(filename, flags=cv2.IMREAD_UNCHANGED):
    """Always read with IMREAD_UNCHANGED to preserve 4 channels"""
    return _original_imread(filename, cv2.IMREAD_UNCHANGED)

patches.imread = imread_4ch
print("Patched imread for 4-channel support")

from ultralytics import YOLO
print("YOLO imported")

In [ ]:
# =============================================================================
# Cell 6: Custom Trainer & Validator with 4-Channel Support
# =============================================================================
from ultralytics.models.yolo.detect import DetectionTrainer, DetectionValidator

def convert_conv_to_4ch(conv_layer):
    """Convert a conv layer from 3ch to 4ch input"""
    if conv_layer.in_channels == 4:
        return conv_layer
    
    new_conv = nn.Conv2d(
        in_channels=4,
        out_channels=conv_layer.out_channels,
        kernel_size=conv_layer.kernel_size,
        stride=conv_layer.stride,
        padding=conv_layer.padding,
        bias=conv_layer.bias is not None
    )
    
    with torch.no_grad():
        # Copy RGB weights
        new_conv.weight[:, :3, :, :] = conv_layer.weight.clone()
        # Initialize depth channel as mean of RGB
        new_conv.weight[:, 3:4, :, :] = conv_layer.weight.mean(dim=1, keepdim=True)
        if conv_layer.bias is not None:
            new_conv.bias = nn.Parameter(conv_layer.bias.clone())
    
    return new_conv


def ensure_model_4ch(model):
    """Ensure model has 4-channel input"""
    try:
        # For AutoBackend wrapped models
        if hasattr(model, 'model') and hasattr(model.model, 'model'):
            first_conv = model.model.model[0].conv
            if first_conv.in_channels == 3:
                print("[4ch] Converting AutoBackend model...")
                model.model.model[0].conv = convert_conv_to_4ch(first_conv)
                return True
        # For direct DetectionModel
        elif hasattr(model, 'model') and hasattr(model.model[0], 'conv'):
            first_conv = model.model[0].conv
            if first_conv.in_channels == 3:
                print("[4ch] Converting DetectionModel...")
                model.model[0].conv = convert_conv_to_4ch(first_conv)
                return True
    except Exception as e:
        print(f"[4ch] Warning: {e}")
    return False


class RGBD4ChValidator(DetectionValidator):
    """Validator that converts model to 4ch before validation"""
    
    def setup_model(self):
        """Override: after model is loaded, convert to 4ch"""
        super().setup_model()
        if self.model is not None:
            ensure_model_4ch(self.model)
            print(f"[Validator] Model 4ch ready")


class RGBD4ChTrainer(DetectionTrainer):
    """Trainer that converts model to 4-channel"""
    
    def setup_model(self):
        """Setup model then convert to 4-channel"""
        super().setup_model()
        
        first_conv = self.model.model[0].conv
        
        if first_conv.in_channels == 4:
            print("[Trainer] Model already 4-channel")
            return
        
        print(f"[Trainer] Converting to 4-channel...")
        self.model.model[0].conv = convert_conv_to_4ch(first_conv)
        print(f"[Trainer] Converted! Shape: {self.model.model[0].conv.weight.shape}")
    
    def get_validator(self):
        """Return custom validator"""
        self.loss_names = "box_loss", "cls_loss", "dfl_loss"
        return RGBD4ChValidator(
            self.test_loader, 
            save_dir=self.save_dir, 
            args=self.args,
            _callbacks=self.callbacks
        )

print("Custom Trainer & Validator defined")

In [ ]:
# =============================================================================
# Cell 7: Training Configuration
# =============================================================================
EXP_PREFIX = "exp_a4b_rgbd_synth"
SEEDS = [42, 123, 456, 789, 101]
EPOCHS = 100
IMGSZ = 640
BATCH_SIZE = 16
DEVICE = 0 if torch.cuda.is_available() else 'cpu'

print(f"Experiment: A.4b RGB+Synthetic Depth (4-Channel) - FIXED")
print(f"Seeds: {SEEDS}")
print(f"Epochs: {EPOCHS}")
print(f"Batch: {BATCH_SIZE}")
print(f"Device: {DEVICE}")

In [ ]:
# =============================================================================
# Cell 8: Training Loop with Custom Trainer
# =============================================================================
results_all = {}

for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"TRAINING A.4b RGBD SYNTHETIC - Seed {seed} ({SEEDS.index(seed)+1}/{len(SEEDS)})")
    print(f"{'='*60}\n")
    
    # Use custom trainer for 4-channel support
    trainer = RGBD4ChTrainer(overrides={
        'model': 'yolo11n.pt',
        'data': str(config_path),
        'imgsz': IMGSZ,
        'epochs': EPOCHS,
        'batch': BATCH_SIZE,
        'device': DEVICE,
        'seed': seed,
        'name': f"{EXP_PREFIX}_seed{seed}",
        'project': str(RUNS_PATH),
        'exist_ok': True,
        'pretrained': True,
        'patience': 30,
        'val': True,
        # Disable HSV augmentation (not applicable for depth)
        'hsv_h': 0.0,
        'hsv_s': 0.0,
        'hsv_v': 0.0,
        'degrees': 0.0,
        'translate': 0.1,
        'scale': 0.5,
        'fliplr': 0.5,
        'mosaic': 1.0,
        'mixup': 0.0,
        'copy_paste': 0.0,
        'erasing': 0.0,
    })
    
    trainer.train()
    
    results_all[seed] = {
        'model_path': str(RUNS_PATH / f"{EXP_PREFIX}_seed{seed}" / "weights" / "best.pt"),
    }
    
    print(f"\nSeed {seed} completed!")
    
    del trainer
    torch.cuda.empty_cache()

print("\n" + "="*60)
print("ALL TRAINING COMPLETED!")
print("="*60)

In [ ]:
# =============================================================================
# Cell 9: Evaluation on Test Set
# =============================================================================
results_dict = {}

print("\n" + "="*60)
print("EVALUATION ON TEST SET")
print("="*60)

for seed in SEEDS:
    model_path = RUNS_PATH / f"{EXP_PREFIX}_seed{seed}" / "weights" / "best.pt"
    
    if not model_path.exists():
        print(f"Model not found: {model_path}")
        continue
    
    print(f"\nSeed {seed}:")
    
    model = YOLO(str(model_path))
    
    # Convert to 4ch if needed
    first_conv = model.model.model[0].conv
    if first_conv.in_channels == 3:
        print("  Converting to 4ch...")
        model.model.model[0].conv = convert_conv_to_4ch(first_conv)
    
    # Verify 4-channel
    print(f"  Model input channels: {model.model.model[0].conv.in_channels}")
    
    metrics = model.val(
        data=str(config_path),
        split="test",
        device=DEVICE,
        name=f"test_{EXP_PREFIX}_seed{seed}",
        exist_ok=True,
    )
    
    results_dict[seed] = {
        'mAP50': metrics.box.map50,
        'mAP50-95': metrics.box.map,
        'Precision': metrics.box.mp,
        'Recall': metrics.box.mr
    }
    
    print(f"  mAP50: {metrics.box.map50:.3f}, mAP50-95: {metrics.box.map:.3f}")
    
    del model
    torch.cuda.empty_cache()

In [ ]:
# =============================================================================
# Cell 10: Results Summary
# =============================================================================
if results_dict:
    df = pd.DataFrame(results_dict).T
    df.index.name = 'Seed'
    avg = df.mean()
    std = df.std()

    print("\n" + "="*60)
    print("A.4b RGB+SYNTHETIC DEPTH (4-CH) - FINAL RESULTS")
    print("="*60 + "\n")
    print(df.to_string(float_format=lambda x: f"{x:.3f}"))

    print("\n" + "-"*60)
    print("SUMMARY (Mean +/- Std)")
    print("-"*60)
    for col in df.columns:
        print(f"  {col}: {avg[col]:.3f} +/- {std[col]:.3f}")

In [ ]:
# =============================================================================
# Cell 11: Save Results
# =============================================================================
output_file = KAGGLE_OUTPUT / 'a4b_rgbd_synthetic_results.txt'

with open(output_file, 'w') as f:
    f.write("="*60 + "\n")
    f.write("A.4b RGB+Synthetic Depth (4-Channel) Results - FIXED\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("Training: epochs=100, patience=30, other=default\n")
    f.write(f"Seeds: {SEEDS}\n")
    f.write("4-Channel Support: YES (Custom Trainer)\n")
    f.write("="*60 + "\n\n")
    f.write("Per-Seed Results:\n")
    f.write(df.to_string(float_format=lambda x: f"{x:.3f}"))
    f.write("\n\n" + "-"*60 + "\n")
    f.write("Summary (Mean +/- Std):\n")
    for col in df.columns:
        f.write(f"  {col}: {avg[col]:.3f} +/- {std[col]:.3f}\n")

print(f"Results saved: {output_file}")

In [ ]:
# =============================================================================
# Cell 12: Create Archives
# =============================================================================
if RUNS_PATH.exists():
    shutil.make_archive(str(BASE_PATH / 'a4b_runs'), 'zip', RUNS_PATH)
    print(f"a4b_runs.zip: {os.path.getsize(str(BASE_PATH / 'a4b_runs.zip'))/1024/1024:.1f} MB")

shutil.make_archive(str(BASE_PATH / 'a4b_output'), 'zip', KAGGLE_OUTPUT)
print("a4b_output.zip created")

print("\nDownload from Output tab:")
print("  - a4b_runs.zip (training runs)")
print("  - a4b_output.zip (results)")